In [0]:
# Load LSTM network and generate text
import sys
import os
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from google.colab import drive

Using TensorFlow backend.


In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Cargar Texto

In [0]:
filename = '/content/drive/Shared drives/test/allSongs.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()[:30150]

#Word Embedding

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  30150
Total Vocab:  28


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 300
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
#print(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  29850


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

#Training

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(512))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="/content/drive/Shared drives/test/weights/weights-improvement-last-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=1, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=80, batch_size=64, callbacks=callbacks_list)

Epoch 1/80
30000/30000 [==============================] - 239s 8ms/step - loss: 2.8784
Epoch 2/80
30000/30000 [==============================] - 237s 8ms/step - loss: 2.7728
Epoch 3/80
30000/30000 [==============================] - 236s 8ms/step - loss: 2.6956
Epoch 4/80
30000/30000 [==============================] - 231s 8ms/step - loss: 2.6330
Epoch 5/80
30000/30000 [==============================] - 232s 8ms/step - loss: 2.5792
Epoch 6/80
30000/30000 [==============================] - 231s 8ms/step - loss: 2.4883
Epoch 7/80
30000/30000 [==============================] - 234s 8ms/step - loss: 2.3686
Epoch 8/80
30000/30000 [==============================] - 237s 8ms/step - loss: 2.2562
Epoch 9/80
30000/30000 [==============================] - 236s 8ms/step - loss: 2.1693
Epoch 10/80
30000/30000 [==============================] - 236s 8ms/step - loss: 2.0997
Epoch 11/80
30000/30000 [==============================] - 237s 8ms/step - loss: 2.0121
Epoch 12/80
30000/30000 [================

#Load weights

In [0]:
filename = "/content/drive/Shared drives/test/weights/weights-improvement-last-29-0.7699.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

#Seed

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

"  cualquier argumento
no le importa si su destino es violento
va tranquila la bala no tiene sentimientos
como un secreto que no quieres escuchar
la bala va dicindolo todo sin hablar
sin levantar sospecha asegura su matanza
por eso tiene llena de plomo su panza
para llegar a su presa no necesita ojos
 "


#Generate

In [0]:
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]

y ms cuando el camino se po te pe estua t mos cabilando alu tnamas contamas cuenqoseal contr aonuellos
los paruido sodos los mieas s senas son la bira

en mi cabeza tin ducilano de ror puestro con sarario muertro cuando te me cinvral
con el ciruo don iosirade gorua de la mlna
ce lo que sodre
el moeo
cono un fuera
s te na empujamo monga sin margarina
y se la empujamo monga sin margarina
y se la empujamo monga sin margarina
y se la empujamos monga bien monga

yoy soy lo que mo esca me ealelina por las matal s ce tenter en un par de asrelloo
so se penue en eamon domo un fuera

el mamon lo que suere
es por que algunan por uue caricndo en ca ciroo an como eo agua bendita
es ltuier areo eo mo henerina parue no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque no le duela
paque n